In [1]:

import sys
import datetime
sys.path.append('/Users/joegradone/SynologyDrive/Drive/Rutgers/Research/code/GitHub/ECCOv4-py')
import ecco_v4_py as ecco
#from ecco_download import *
import numpy as np
import pandas as pd
from os.path import join,expanduser
import xarray as xr
import matplotlib.pyplot as plt
from getpass import getpass
from http.cookiejar import CookieJar
from pathlib import Path
from netrc import netrc
from cartopy.mpl.geoaxes import GeoAxes
import cartopy
import geopandas
from scipy.stats import linregress
from mpl_toolkits.axes_grid1 import AxesGrid
# library to download files
from urllib import request
from scipy import signal
import glob
import xgcm
import scipy.signal as sps
import scipy.linalg as spl
from xgcm import Grid
import cmocean.cm as cmo

In [2]:
download_root_dir = Path('/Users/joegradone/SynologyDrive/Drive/Rutgers/Research/data/ECCO/')
## Load grid
ecco_grid = xr.open_dataset('/Users/joegradone/SynologyDrive/Drive/Rutgers/Research/data/ECCO/ECCO_L4_GEOMETRY_LLC0090GRID_V4R4/GRID_GEOMETRY_ECCO_V4r4_native_llc0090.nc')
ecco_grid = ecco_grid.isel(tile=[10,11])
grid = xgcm.Grid(ecco_grid)

In [3]:
fnames = glob.glob(''.join([str(download_root_dir),'/ECCO_L4_OCEAN_BOLUS_STREAMFUNCTION_LLC0090GRID_DAILY_V4R4/*.nc']))
bolus = xr.open_mfdataset(fnames, parallel=True, data_vars='minimal', coords='minimal', compat='override')

In [4]:
fnames = glob.glob(''.join([str(download_root_dir),'/ECCO_L4_OCEAN_3D_VOLUME_FLUX_LLC0090GRID_DAILY_V4R4/*.nc']))
vol_flux = xr.open_mfdataset(fnames, parallel=True, data_vars='minimal', coords='minimal', compat='override')

In [5]:
fnames = glob.glob(''.join([str(download_root_dir),'/ECCO_L4_TEMP_SALINITY_LLC0090GRID_DAILY_V4R4/*.nc']))
ts = xr.open_mfdataset(fnames, parallel=True, data_vars='minimal', coords='minimal', compat='override')

In [6]:
fnames = glob.glob(''.join([str(download_root_dir),'/ECCO_L4_FRESH_FLUX_LLC0090GRID_DAILY_V4R4/*.nc']))
fresh_flux = xr.open_mfdataset(fnames, parallel=True, data_vars='minimal', coords='minimal', compat='override')

In [7]:
# set values on land to zero (instead of NaN)
bolus['GM_PsiX'] = bolus.GM_PsiX.where(ecco_grid.hFacW.values > 0,0)
bolus['GM_PsiY'] = bolus.GM_PsiY.where(ecco_grid.hFacS.values > 0,0)

#grid = ecco.get_llc_grid(ecco_grid)

UVELSTAR = grid.diff(bolus.GM_PsiX, 'Z', boundary='fill')/ecco_grid.drF
VVELSTAR = grid.diff(bolus.GM_PsiY, 'Z', boundary='fill')/ecco_grid.drF

GM_PsiXY_diff = grid.diff_2d_vector({'X' : bolus.GM_PsiX*ecco_grid.dyG,
                                     'Y' : bolus.GM_PsiY*ecco_grid.dxG}, boundary = 'fill')
WVELSTAR = (GM_PsiXY_diff['X'] + GM_PsiXY_diff['Y'])/ecco_grid.rA

SALT_at_u = grid.interp(ts.SALT, 'X', boundary='extend')
SALT_at_v = grid.interp(ts.SALT, 'Y', boundary='extend')
SALT_at_w = grid.interp(ts.SALT, 'Z', boundary='extend')

# Remove oceFWflx from WVELMASS
WVELMASS = vol_flux.WVELMASS.transpose('time','tile','k_l','j','i')
oceFWflx = fresh_flux.oceFWflx.assign_coords(k_l=0).expand_dims('k_l').transpose('time','tile','k_l','j','i')

# Seawater density (kg/m^3)
rhoconst = 1029

oceFWflx = (oceFWflx/rhoconst)
WVELMASS = xr.concat([WVELMASS.sel(k_l=0) + oceFWflx, WVELMASS[:,:,1:]],
                     dim='k_l').transpose('time','tile','k_l','j','i')

# Salinity advective (Eulerian+Bolus) fluxes (psu m^3/s)
ADVx_SLT = (vol_flux.UVELMASS+UVELSTAR)*ecco_grid.dyG*ecco_grid.drF*SALT_at_u

ADVy_SLT = (vol_flux.VVELMASS+VVELSTAR)*ecco_grid.dxG*ecco_grid.drF*SALT_at_v

ADVr_SLT = (WVELMASS+WVELSTAR)*ecco_grid.rA*SALT_at_w
ADVr_SLT = ADVr_SLT.chunk({'time':1,'tile':13,'k_l':50,'j':90,'i':90})

ADVxy_diff = grid.diff_2d_vector({'X' : ADVx_SLT, 'Y' : ADVy_SLT}, boundary = 'fill')
adv_hConvS = (-(ADVxy_diff['X'] + ADVxy_diff['Y']))
adv_vConvS = grid.diff(ADVr_SLT, 'Z', boundary='fill')


In [8]:
ADVx_SLT_bolus = (UVELSTAR)*ecco_grid.dyG*ecco_grid.drF*SALT_at_u
ADVx_SLT_euler = (vol_flux.UVELMASS)*ecco_grid.dyG*ecco_grid.drF*SALT_at_u

ADVy_SLT_bolus = (UVELSTAR)*ecco_grid.dyG*ecco_grid.drF*SALT_at_v
ADVy_SLT_euler = (vol_flux.UVELMASS)*ecco_grid.dyG*ecco_grid.drF*SALT_at_v

ADVr_SLT_bolus = (WVELSTAR)*ecco_grid.rA*SALT_at_w
ADVr_SLT_euler = (WVELMASS)*ecco_grid.rA*SALT_at_w


In [9]:
new_grid_delta_lat = 1
new_grid_delta_lon = 1

new_grid_min_lat = -10
new_grid_max_lat = 35

new_grid_min_lon = -100
new_grid_max_lon = -37


lon_shape = len(np.arange(new_grid_min_lon,new_grid_max_lon,new_grid_delta_lon))
lat_shape = len(np.arange(new_grid_min_lat,new_grid_max_lat,new_grid_delta_lat))

depth_len = 23 # 29 for 1000 m, 23 for 500 m

ADVx_SLT_bolus_interp = np.empty([lat_shape,lon_shape,depth_len,len(ADVx_SLT_bolus.time)])
ADVx_SLT_bolus_interp[:] = np.nan
ADVx_SLT_euler_interp = np.empty([lat_shape,lon_shape,depth_len,len(ADVx_SLT_bolus.time)])
ADVx_SLT_euler_interp[:] = np.nan

ADVy_SLT_bolus_interp = np.empty([lat_shape,lon_shape,depth_len,len(ADVx_SLT_bolus.time)])
ADVy_SLT_bolus_interp[:] = np.nan
ADVy_SLT_euler_interp = np.empty([lat_shape,lon_shape,depth_len,len(ADVx_SLT_bolus.time)])
ADVy_SLT_euler_interp[:] = np.nan

ADVr_SLT_bolus_interp = np.empty([lat_shape,lon_shape,depth_len,len(ADVx_SLT_bolus.time)])
ADVr_SLT_bolus_interp[:] = np.nan
ADVr_SLT_euler_interp = np.empty([lat_shape,lon_shape,depth_len,len(ADVx_SLT_bolus.time)])
ADVr_SLT_euler_interp[:] = np.nan


for k in np.arange(0,depth_len):
    for t in np.arange(0,len(ADVx_SLT_bolus.time)):
        new_grid_lon, new_grid_lat, _, _, ADVx_SLT_bolus_interp[:,:,k,t] = ecco.resample_to_latlon(ecco_grid.XC.values,
                                     ecco_grid.YC.values,
                                     ADVx_SLT_bolus.isel(time=t,k=k).compute(),
                                     new_grid_min_lat, new_grid_max_lat, new_grid_delta_lat,
                                     new_grid_min_lon, new_grid_max_lon, new_grid_delta_lon,
                                     fill_value = np.NaN,
                                     mapping_method = 'nearest_neighbor',
                                     radius_of_influence = 120000)

    print('k',k)
    print(datetime.datetime.now())


k 0
2024-06-22 04:35:55.918486



KeyboardInterrupt



# TRY THIS IN SPYDER IF THIS DOESNT WORK